In [2]:
from __future__ import absolute_import,division,print_function,unicode_literals
import os
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import cProfile

# Basic of TF2.0

In [3]:
tf.executing_eagerly()

True

In [7]:
a=tf.constant([[1,2],[3,4]])
print(a)
#Broadcasting
b=tf.add(a,1)
print(b)
#element-wise multiplication
print(a*b)
import numpy as np
c = np.multiply(a,b)
print(c)
#multiplication of matrices
print(tf.matmul(a,b))
#transfer a tensor to numpy array
print(a.numpy())

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)
[[ 2  6]
 [12 20]]
tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)
[[1 2]
 [3 4]]


# Train a model

## 1. load data

In [10]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

x_train = tf.cast(x_train[...,tf.newaxis]/255,tf.float32)
x_test= tf.cast(x_test[...,tf.newaxis]/255,tf.float32)
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

## 2.Build model (CNN)

### 2.1 Build the model using Sequential

In [15]:
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [16]:
mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

### 2.2 Build the model using Model

In [146]:
inputs = tf.keras.Input(shape=(None,None,1),name="digits")
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(inputs)
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10,activation="softmax")(ave_pool)
mnist_model_2 = tf.keras.Model(inputs=inputs,outputs=outputs)

In [147]:
mnist_model_2.summary()

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, None, None, 16)    160       
_________________________________________________________________
conv2d_87 (Conv2D)           (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d_13  (None, 16)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


## 3.Training methods

### 3.1 Use keras fit

In [31]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                   validation_split=0.1,shuffle=True,
                   loss=tf.keras.losses.categorical_crossentropy,
                   metrics=["accuracy"])

In [64]:
mnist_model.fit(x_train,y_train,batch_size=128,epochs=50)

Train on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 15s 2ms/sample - loss: 1.7994 - accuracy: 0.4904
Epoch 2/50
10000/10000 [==============================] - 15s 2ms/sample - loss: 1.6741 - accuracy: 0.5256
Epoch 3/50
10000/10000 [==============================] - 16s 2ms/sample - loss: 1.5353 - accuracy: 0.5578
Epoch 4/50
10000/10000 [==============================] - 17s 2ms/sample - loss: 1.3955 - accuracy: 0.5991
Epoch 5/50
10000/10000 [==============================] - 16s 2ms/sample - loss: 1.2759 - accuracy: 0.6238
Epoch 6/50
10000/10000 [==============================] - 17s 2ms/sample - loss: 1.1814 - accuracy: 0.6449
Epoch 7/50
10000/10000 [==============================] - 17s 2ms/sample - loss: 1.0891 - accuracy: 0.6744
Epoch 8/50
10000/10000 [==============================] - 16s 2ms/sample - loss: 1.0111 - accuracy: 0.6859
Epoch 9/50
10000/10000 [==============================] - 17s 2ms/sample - loss: 0.9512 - accuracy: 0.7052
Epoch 10/50
10

In [65]:
train_pred = mnist_model.predict(x_train)
train_acc = sum(np.where(train_pred.argmax(axis=1)==y_train.argmax(axis=1),1,0))/y_train.shape[0]
print("train_acc=",train_acc)
test_pred = mnist_model.predict(x_test)
test_acc = train_acc = sum(np.where(test_pred.argmax(axis=1)==y_test.argmax(axis=1),1,0))/y_test.shape[0]
print("test_acc=",test_acc)

train_acc= 0.9176
test_acc= 0.888


In [148]:
mnist_model_2.compile(loss = tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

In [152]:
mnist_model_2.fit(x_train,y_train,batch_size=128,epochs=30)

Train on 10000 samples
Epoch 1/30
10000/10000 [==============================] - 6s 566us/sample - loss: 1.5770 - accuracy: 0.4396
Epoch 2/30
10000/10000 [==============================] - 6s 598us/sample - loss: 1.5504 - accuracy: 0.4501
Epoch 3/30
10000/10000 [==============================] - 6s 595us/sample - loss: 1.5279 - accuracy: 0.4580
Epoch 4/30
10000/10000 [==============================] - 6s 580us/sample - loss: 1.5018 - accuracy: 0.4715
Epoch 5/30
10000/10000 [==============================] - 6s 615us/sample - loss: 1.4854 - accuracy: 0.4789
Epoch 6/30
10000/10000 [==============================] - 5s 547us/sample - loss: 1.4615 - accuracy: 0.4933
Epoch 7/30
10000/10000 [==============================] - 6s 553us/sample - loss: 1.4441 - accuracy: 0.4973
Epoch 8/30
10000/10000 [==============================] - 6s 602us/sample - loss: 1.4271 - accuracy: 0.5077
Epoch 9/30
10000/10000 [==============================] - 6s 587us/sample - loss: 1.4116 - accuracy: 0.5135
Epoch

In [154]:
train_pred = mnist_model_2.predict(x_train)
train_acc = sum(np.where(train_pred.argmax(axis=1)==y_train.argmax(axis=1),1,0))/y_train.shape[0]
print("train_acc=",train_acc)
test_pred = mnist_model_2.predict(x_test)
test_acc = train_acc = sum(np.where(test_pred.argmax(axis=1)==y_test.argmax(axis=1),1,0))/y_test.shape[0]
print("test_acc=",test_acc)

train_acc= 0.6311
test_acc= 0.616


### 3.2 Use TF2.0

In [197]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [198]:
x_train = tf.cast(x_train[...,tf.newaxis]/255, tf.float32)
x_test = tf.cast(x_test[...,tf.newaxis]/255, tf.float32)

In [199]:
dataset = tf.data.Dataset.from_tensor_slices((x_train,tf.cast(y_train,tf.int64)))

dataset = dataset.shuffle(1000).batch(32)

In [207]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) 
loss_history=[]

In [209]:
for epoch in range(10):
    total_loss = 0
    for (batch,(images,labels)) in enumerate (dataset):
        with tf.GradientTape() as tape:
            logits = mnist_model(images,training=True) #predicted labels
            loss_value = loss(labels,logits)
            total_loss += loss_value
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
    print("Epoch {} finished, loss={}".format(epoch,total_loss/1000))

Epoch 0 finished, loss=0.07374794036149979
Epoch 1 finished, loss=0.07220444828271866
Epoch 2 finished, loss=0.07176249474287033
Epoch 3 finished, loss=0.07087982445955276
Epoch 4 finished, loss=0.07136518508195877
Epoch 5 finished, loss=0.07041977345943451
Epoch 6 finished, loss=0.0707215741276741
Epoch 7 finished, loss=0.06859298050403595
Epoch 8 finished, loss=0.07042422890663147
Epoch 9 finished, loss=0.0667891651391983


In [211]:
train_pred = mnist_model.predict(x_train)
train_acc = sum(np.where(train_pred.argmax(axis=1)==y_train,1,0))/y_train.shape[0]
print("train_acc=",train_acc)
test_pred = mnist_model.predict(x_test)
test_acc = train_acc = sum(np.where(test_pred.argmax(axis=1)==y_test,1,0))/y_test.shape[0]
print("test_acc=",test_acc)

train_acc= 0.9676
test_acc= 0.942
